In [1]:
import pandas as pd
train = pd.read_csv("./train.csv")
X = (train.iloc[:,1:].values).astype('float32') / 255.0
Y = train.iloc[:,0].values.astype('int32')
X = X.reshape(X.shape[0], 28, 28, 1)

from keras.utils.np_utils import to_categorical
Y = to_categorical(Y)

from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.1, random_state=42)

print ("number of training examples = " + str(X_train.shape[0]))
print ("number of test examples = " + str(X_test.shape[0]))
print ("X_train shape: " + str(X_train.shape))
print ("Y_train shape: " + str(Y_train.shape))
print ("X_test shape: " + str(X_test.shape))
print ("Y_test shape: " + str(Y_test.shape))

Using TensorFlow backend.
/usr/local/Cellar/python3/3.6.2/Frameworks/Python.framework/Versions/3.6/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


number of training examples = 37800
number of test examples = 4200
X_train shape: (37800, 28, 28, 1)
Y_train shape: (37800, 10)
X_test shape: (4200, 28, 28, 1)
Y_test shape: (4200, 10)


In [2]:
test = pd.read_csv("./test.csv")
X_score = (test.values).astype('float32')/255.0
X_score = X_score.reshape(X_score.shape[0], 28, 28, 1)
print(X_score.shape)

(28000, 28, 28, 1)


In [3]:
import math
import numpy as np

# GRADED FUNCTION: random_mini_batches
def random_mini_batches(X, Y, mini_batch_size = 100, seed = 0):
    """
    Creates a list of random minibatches from (X, Y)
    
    Arguments:
    X -- input data, of shape (input size, number of examples)
    Y -- true "label" vector (1 for blue dot / 0 for red dot), of shape (1, number of examples)
    mini_batch_size -- size of the mini-batches, integer
    
    Returns:
    mini_batches -- list of synchronous (mini_batch_X, mini_batch_Y)
    """
    
    np.random.seed(seed)            # To make your "random" minibatches the same as ours
    m = X.shape[0]                  # number of training examples
    mini_batches = []
        
    # Step 1: Shuffle (X, Y)
    permutation = list(np.random.permutation(m))
    shuffled_X = X[permutation]
    shuffled_Y = Y[permutation]

    # Step 2: Partition (shuffled_X, shuffled_Y). Minus the end case.
    num_complete_minibatches = math.floor(m/mini_batch_size) # number of mini batches of size mini_batch_size in your partitionning
    for k in range(0, num_complete_minibatches):
        ### START CODE HERE ### (approx. 2 lines)
        mini_batch_X = shuffled_X[k*mini_batch_size:(k+1)*mini_batch_size]
        mini_batch_Y = shuffled_Y[k*mini_batch_size:(k+1)*mini_batch_size]
        ### END CODE HERE ###
        mini_batch = (mini_batch_X, mini_batch_Y)
        mini_batches.append(mini_batch)
    
    # Handling the end case (last mini-batch < mini_batch_size)
    if m % mini_batch_size != 0:
        ### START CODE HERE ### (approx. 2 lines)
        mini_batch_X = shuffled_X[num_complete_minibatches*mini_batch_size:]
        mini_batch_Y = shuffled_Y[num_complete_minibatches*mini_batch_size:]
        ### END CODE HERE ###
        mini_batch = (mini_batch_X, mini_batch_Y)
        mini_batches.append(mini_batch)
    
    return mini_batches

In [4]:
"""A deep MNIST classifier using convolutional layers.
See extensive documentation at
https://www.tensorflow.org/get_started/mnist/pros
"""
# Disable linter warnings to maintain consistency with tutorial.
# pylint: disable=invalid-name
# pylint: disable=g-bad-import-order

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import argparse
import sys
import tempfile

import tensorflow as tf

FLAGS = None


def deepnn(x_image):
  """deepnn builds the graph for a deep net for classifying digits.
  Args:
    x: an input tensor with the dimensions (N_examples, 784), where 784 is the
    number of pixels in a standard MNIST image.
  Returns:
    A tuple (y, keep_prob). y is a tensor of shape (N_examples, 10), with values
    equal to the logits of classifying the digit into one of 10 classes (the
    digits 0-9). keep_prob is a scalar placeholder for the probability of
    dropout.
  """
  # Reshape to use within a convolutional neural net.
  # Last dimension is for "features" - there is only one here, since images are
  # grayscale -- it would be 3 for an RGB image, 4 for RGBA, etc.
#   with tf.name_scope('reshape'):
#     x_image = tf.reshape(x, [-1, 28, 28, 1])

  # First convolutional layer - maps one grayscale image to 32 feature maps.
  with tf.name_scope('conv1'):
    W_conv1 = weight_variable([5, 5, 1, 32])
    b_conv1 = bias_variable([32])
    h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)

  # Pooling layer - downsamples by 2X.
  with tf.name_scope('pool1'):
    h_pool1 = max_pool_2x2(h_conv1)

  # Second convolutional layer -- maps 32 feature maps to 64.
  with tf.name_scope('conv2'):
    W_conv2 = weight_variable([5, 5, 32, 64])
    b_conv2 = bias_variable([64])
    h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)

  # Second pooling layer.
  with tf.name_scope('pool2'):
    h_pool2 = max_pool_2x2(h_conv2)

  # Fully connected layer 1 -- after 2 round of downsampling, our 28x28 image
  # is down to 7x7x64 feature maps -- maps this to 1024 features.
  with tf.name_scope('fc1'):
    W_fc1 = weight_variable([7 * 7 * 64, 1024])
    b_fc1 = bias_variable([1024])

    h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*64])
    h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)

  # Dropout - controls the complexity of the model, prevents co-adaptation of
  # features.
  with tf.name_scope('dropout'):
    keep_prob = tf.placeholder(tf.float32)
    h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

  # Map the 1024 features to 10 classes, one for each digit
  with tf.name_scope('fc2'):
    W_fc2 = weight_variable([1024, 10])
    b_fc2 = bias_variable([10])

    y_conv = tf.matmul(h_fc1_drop, W_fc2) + b_fc2
  return y_conv, keep_prob


def conv2d(x, W):
  """conv2d returns a 2d convolution layer with full stride."""
  return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')


def max_pool_2x2(x):
  """max_pool_2x2 downsamples a feature map by 2X."""
  return tf.nn.max_pool(x, ksize=[1, 2, 2, 1],
                        strides=[1, 2, 2, 1], padding='SAME')


def weight_variable(shape):
  """weight_variable generates a weight variable of a given shape."""
  initial = tf.truncated_normal(shape, stddev=0.1)
  return tf.Variable(initial)


def bias_variable(shape):
  """bias_variable generates a bias variable of a given shape."""
  initial = tf.constant(0.1, shape=shape)
  return tf.Variable(initial)


def run_network(num_epochs = 25, minibatch_size = 50):
  # Create the model
  x = tf.placeholder(tf.float32, [None, 28, 28, 1])

  # Define loss and optimizer
  y_ = tf.placeholder(tf.float32, [None, 10])

  # Build the graph for the deep net
  y_conv, keep_prob = deepnn(x)

  with tf.name_scope('loss'):
    cross_entropy = tf.nn.softmax_cross_entropy_with_logits(labels=y_,
                                                            logits=y_conv)
  cross_entropy = tf.reduce_mean(cross_entropy)

  with tf.name_scope('adam_optimizer'):
    train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)

  with tf.name_scope('accuracy'):
    correct_prediction = tf.equal(tf.argmax(y_conv, 1), tf.argmax(y_, 1))
    correct_prediction = tf.cast(correct_prediction, tf.float32)
  accuracy = tf.reduce_mean(correct_prediction)

#   graph_location = tempfile.mkdtemp()
#   print('Saving graph to: %s' % graph_location)
#   train_writer = tf.summary.FileWriter(graph_location)
#   train_writer.add_graph(tf.get_default_graph())

  seed = 3
  m = X_train.shape[0]
  with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for i in range(num_epochs):
        seed = seed + 1
        minibatches = random_mini_batches(X_train, Y_train, minibatch_size, seed)
        
        j = 0
        for minibatch in minibatches:
            (minibatch_X, minibatch_Y) = minibatch
            train_step.run(feed_dict={x: minibatch_X, y_:minibatch_Y, keep_prob: 0.5})
        
            j = j+1
            if j % 100 == 0:
                train_accuracy = accuracy.eval(feed_dict={x: minibatch_X, y_: minibatch_Y, keep_prob: 1.0})
                print('step %d, batch %d, training accuracy %g' % (i, j, train_accuracy))
            
    print('test accuracy %g' % accuracy.eval(feed_dict={
      x: X_test, y_: Y_test, keep_prob: 1.0}))
    
    predict_op = tf.argmax(y_conv, 1)
    predictions = []
    for i in range(int(X_score.shape[0]/50)):
        p = sess.run(predict_op, {x: X_score[i*50:(i+1)*50], keep_prob: 1.0})
        predictions.extend(p)
    
    return predictions

In [5]:
predictions = run_network()

step 0, batch 100, training accuracy 0.88
step 0, batch 200, training accuracy 0.9
step 0, batch 300, training accuracy 0.94
step 0, batch 400, training accuracy 0.96
step 0, batch 500, training accuracy 0.96
step 0, batch 600, training accuracy 0.96
step 0, batch 700, training accuracy 0.98
step 1, batch 100, training accuracy 0.98
step 1, batch 200, training accuracy 0.96
step 1, batch 300, training accuracy 0.96
step 1, batch 400, training accuracy 1
step 1, batch 500, training accuracy 0.98
step 1, batch 600, training accuracy 0.96
step 1, batch 700, training accuracy 1
step 2, batch 100, training accuracy 0.98
step 2, batch 200, training accuracy 0.98
step 2, batch 300, training accuracy 0.98
step 2, batch 400, training accuracy 0.98
step 2, batch 500, training accuracy 1
step 2, batch 600, training accuracy 1
step 2, batch 700, training accuracy 0.98
step 3, batch 100, training accuracy 0.96
step 3, batch 200, training accuracy 1
step 3, batch 300, training accuracy 0.98
step 3, 

In [7]:
print(len(predictions))
submissions=pd.DataFrame({"ImageId": list(range(1,len(predictions)+1)),
                         "Label": predictions})
submissions.to_csv("tf3.csv", index=False, header=True)

28000
